In [10]:
#只需要添加nn.BatchNorm层并指定对二维卷积的通道(axis=1)进行批量归一化
from mxnet.gluon import nn

net = nn.Sequential()
with net.name_scope():
    # 第一层卷积
    net.add(nn.Conv2D(channels=20, kernel_size=5))
    ### 添加了批量归一化层
    net.add(nn.BatchNorm(axis=1))
    net.add(nn.Activation(activation='relu'))
    net.add(nn.MaxPool2D(pool_size=2, strides=2))
    # 第二层卷积
    net.add(nn.Conv2D(channels=50, kernel_size=3))
    ### 添加了批量归一化层
    net.add(nn.BatchNorm(axis=1))
    net.add(nn.Activation(activation='relu'))
    net.add(nn.MaxPool2D(pool_size=2, strides=2))
    net.add(nn.Flatten())
    # 第一层全连接
    #net.add(nn.BatchNorm())
    net.add(nn.Dense(128, activation="relu"))
    
    # 第二层全连接
    #net.add(nn.BatchNorm())
    net.add(nn.Dense(10))

In [11]:
import sys
sys.path.append('..')
import utils
from mxnet import autograd
from mxnet import gluon
from mxnet import nd
# from mxnet import init

ctx = utils.try_gpu()
net.initialize(ctx=ctx)

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.2})

for epoch in range(10):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
    test_acc = utils.evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.597842, Train acc 0.777000, Test acc 0.854004
Epoch 1. Loss: 0.365489, Train acc 0.865680, Test acc 0.877832
Epoch 2. Loss: 0.310734, Train acc 0.885755, Test acc 0.890527
Epoch 3. Loss: 0.275221, Train acc 0.899047, Test acc 0.892383
Epoch 4. Loss: 0.257890, Train acc 0.905020, Test acc 0.901855
Epoch 5. Loss: 0.237552, Train acc 0.912511, Test acc 0.899414
Epoch 6. Loss: 0.222687, Train acc 0.916650, Test acc 0.908789
Epoch 7. Loss: 0.208203, Train acc 0.923094, Test acc 0.909668
Epoch 8. Loss: 0.195719, Train acc 0.927161, Test acc 0.912012
Epoch 9. Loss: 0.186561, Train acc 0.930114, Test acc 0.910937
